In [1]:
# import numpy as np

# from toolboxes.inference_toolbox.parameter import Parameter
# from toolboxes.inference_toolbox.likelihood import Likelihood
# from toolboxes.inference_toolbox.model import Model

# I_y = Parameter(name=['I_y','I_z'],prior_select='gaussian',multi_mode=True).add_prior_param(
#     'mu',  [[0,0], [10,10], [20,20]]).add_prior_param(
#         'cov',[[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]])
# I_y.sample_param()



In [2]:
# import numpy as np
# import pandas as pd
# from toolboxes.inference_toolbox.model import Model, add_model, delete_model

# add_model('full_log_gpm','log10(Q/(2*u*pi*I_y*I_z*(x-x_0)**2)*exp(-(y-y_0)**2/(2*I_y**2*(x-x_0)**2))*(exp(-(z-z_0)**2/(2*I_z**2*(x-x_0)**2))+exp(-(z+z_0)**2/(2*I_z**2*(x-x_0)**2))))',
# #            all_param_names=['I_y','I_z','Q','H','u', 'x_0', 'y_0', 'z_0'], independent_variables=['x','y','z'], dependent_variables=['Concentration'])

#     # delete_model('gpm_norm')
# # add_model('curve', 'm*x**2+c',all_param_names=['m','c'], dependent_variables=['y'], independent_variables=['x'])
# model = Model('full_log_gpm').add_fixed_model_param('H', 1).add_fixed_model_param('u', 1).add_fixed_model_param('x_0', 0).add_fixed_model_param('y_0', 1).add_fixed_model_param('z_0', 1)
# model.get_model()(pd.Series({'I_y': 1, 'I_z':2, 'Q':3}), [np.array([1,2,3]), np.array([4,5,6]), np.array([7,8,9])])

In [3]:
# from toolboxes.data_processing_toolbox.raw_data_processor import RawDataProcessor

# import os

# os.chdir('/PhD_project/')

# processor_params = {
#     'experiments_list': [
#         'Exp1',
#         'Exp2',
#         'Exp3',
#         'Exp4',
#         'Exp5',
#         'Exp6',
#         'Exp7',
#         'Exp8',
#         'Exp9',
#         'Exp10',
#         'Exp11',
#         'Exp12',
#     ],
#     'meta_data_select': 'GBR_data_summary',
#     'input_header': 'Concentration',
#     'output_header': 'Concentration',
#     'log_output_data':True,
#     'gridding': [500,500,100]
# }

# train, test = RawDataProcessor('GBR_data', 'GBR_data_gridded_500_500_100', 'GBR_processor', processor_params).process_data()
# train

In [1]:
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.sampler import Sampler
from toolboxes.data_processing_toolbox.sim_data_processor import SimDataProcessor
import numpy as np
import pandas as pd
import os

os.chdir('/PhD_project/')



# Define your main process as a function
sim_model = Model('log_gpm_norm').add_fixed_model_param('H', 1).add_fixed_model_param('I_y',1).add_fixed_model_param('I_z',1).add_fixed_model_param('Q',1e13)
sim_domain = Domain('cone_from_source_z_limited').add_domain_param('r',1000).add_domain_param('theta',np.pi/8).add_domain_param('source', [0,0,1]).add_domain_param('resolution',100)
simulator_params = {
    'output_header': 'Concentration',
    'log_output_data': True,
}

sim_data_processor = SimDataProcessor('sim_data_2', sim_model, sim_domain, simulator_params)

model = Model('log_gpm_norm').add_fixed_model_param('H', 1)
likelihood  = likelihood = Likelihood('gaussian')
I_y_and_I_z = Parameter(name=['I_y','I_z'],prior_select='log_norm',multi_mode=True).add_prior_param('mu',  [[1,1], [1.1,1.1], [1.2,1.2]]).add_prior_param('cov',[[[1,0],[0,1]],[[2,0],[0,2]],[[3,0],[0,3]]])
Q = Parameter(name='Q',prior_select='log_norm').add_prior_param('mu',  1e13).add_prior_param('cov', 1e12)
sigma = Parameter(name='sigma',prior_select='log_norm').add_prior_param('mu',  1).add_prior_param('cov', 1)
inference_params = pd.Series({'I_y_and_I_z': I_y_and_I_z, 'Q': Q, 'sigma':sigma})

samples, chain_samples, fields = Sampler(inference_params, model, likelihood, sim_data_processor, n_samples = 100).sample_all()


/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pstats

# Open the .prof file
profiler = pstats.Stats('profile_results.prof')

# Strip directory names from the function names
profiler.strip_dirs()

# Sort the statistics by cumulative time
profiler.sort_stats('cumulative')

# Print the top 20 functions sorted by cumulative time
profiler.print_stats(20)

# Print a call graph
profiler.print_callers(20)
profiler.print_callees(20)

Mon May 27 01:44:54 2024    profile_results.prof

         11259022 function calls (11237893 primitive calls) in 3.819 seconds

   Ordered by: cumulative time
   List reduced from 988 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     21/1    0.000    0.000    3.819    3.819 {built-in method builtins.exec}
        1    0.000    0.000    3.819    3.819 <string>:1(<module>)
        1    0.000    0.000    3.819    3.819 3525037979.py:17(main)
        1    0.000    0.000    3.657    3.657 sampler.py:14(__init__)
        1    0.000    0.000    3.657    3.657 sim_data_processor.py:131(process_data)
        1    0.000    0.000    3.657    3.657 domain.py:82(create_domain)
        1    0.005    0.005    3.657    3.657 domain.py:282(_create_z_limited_cone)
        1    0.679    0.679    3.653    3.653 domain.py:246(_create_cone)
   403842    0.274    0.000    2.816    0.000 generic.py:6284(__getattr__)
   403751    0.389    0.000    1.434

In [ ]:
import sympy as sp
import json
import os

os.chdir('/PhD_project/')


def string_to_sympy_expr(expression_str, variables, mathematical_words):
    """
    Convert a string representation of a mathematical expression to a SymPy expression.

    Parameters:
    - expression_str (str): The string representation of the expression.
    - variables (dict): A dictionary of SymPy symbols.
    - mathematical_words (dict): A dictionary of mathematical functions and constants.

    Returns:
    - sympy.Expr: The corresponding SymPy expression.
    """
    try:
        expr = sp.sympify(expression_str, locals={**variables, **mathematical_words})
        return expr
    except (sp.SympifyError, ValueError) as e:
        print(f"Error converting string to SymPy expression: {e}")
        return None


# Specify the file path of the model.json file
file_path = '/PhD_project/app/toolboxes/inference_toolbox/models.json'

# Read the contents of the model.json file
with open(file_path, 'r') as file:
    model_data = json.load(file)

model_select = 'full_log_gpm'
# Extract the independent variables, dependent variables, and all parameter names
independent_variables = model_data[model_select]['independent_variables']
dependent_variables = model_data[model_select]['dependent_variables']
all_param_names = model_data[model_select]['all_param_names']
model_string_expr = model_data[model_select]['model']

variables = {}

for param in independent_variables:
    variables[param] = sp.symbols(param)
for param in all_param_names:
    variables[param] = sp.symbols(param)

# Define the mathematical words to be used in sympify
mathematical_words = {
    'pi': sp.pi, 
    'exp': sp.exp, 
    'log': sp.log, 
    'log10': lambda arg: sp.log(arg, 10), 
    'sqrt': sp.sqrt, 
    'sin': sp.sin, 
    'cos': sp.cos, 
    'tan': sp.tan, 
    'asin': sp.asin, 
    'acos': sp.acos, 
    'atan': sp.atan, 
    'sinh': sp.sinh, 
    'cosh': sp.cosh, 
    'tanh': sp.tanh,
    'asinh': sp.asinh, 
    'acosh': sp.acosh, 
    'atanh': sp.atanh
}


# Convert the string to a SymPy expression
sum_expr = string_to_sympy_expr(model_string_expr, variables, mathematical_words)

# Output the SymPy expression
print(sum_expr)

# Evaluate the expression
fixed_model_params = {'u': 1, 'x_0': 0, 'y_0': 1, 'z_0': 1}

inference_param_vars = [variables[param] for param in all_param_names if param not in fixed_model_params]
sum_expr = sum_expr.subs(fixed_model_params)

expt_func = sp.lambdify([variables['x'], variables['y'], variables['z'], *inference_param_vars], sum_expr, 'jax')

expt_func()



log(Q*(exp(-(z + z_0)**2/(2*I_z**2*(x - x_0)**2)) + exp(-(z - z_0)**2/(2*I_z**2*(x - x_0)**2)))*exp(-(y - y_0)**2/(2*I_y**2*(x - x_0)**2))/(2*pi*I_y*I_z*u*(x - x_0)**2))/log(10)


TypeError: _lambdifygenerated() missing 7 required positional arguments: 'x', 'y', 'z', 'I_y', 'I_z', 'Q', and 'H'

In [ ]:
import numpy as np

dict1 = {'a': np.array([1,2,3]), 'b': np.array([4,5,6]), 'c': np.array([7,8,9])}
dict2 = {'a': [1,2,3], 'b': [4,5,6], 'c': [7,8,9]}

dict1_equal_dict2 = {k: list(v) if isinstance(v, np.ndarray) else v for k, v in dict1.items()} == {k: list(v) if isinstance(v, np.ndarray) else v for k, v in dict2.items()}



print(dict1_equal_dict2)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()